In [6]:
import pandas as pd
from io import StringIO
import requests
import tqdm
import tqdm.asyncio
import asyncio

async def getCurie_Disease_async(session, name):
    itemRequest = f'https://name-resolution-sri.renci.org/lookup?string={name}&autocomplete=false&offset=0&limit=10&biolink_type=DiseaseOrPhenotypicFeature'
    async with session.get(itemRequest) as response:
        text = await response.text()
    
    returned = pd.read_json(StringIO(text))
    resolvedName = returned.curie
    resolvedLabel = returned.label
    return resolvedName, resolvedLabel

def getCurie_Disease(name):
    itemRequest = 'https://name-resolution-sri.renci.org/lookup?string=' + name + '&autocomplete=false&offset=0&limit=10&biolink_type=DiseaseOrPhenotypicFeature'
    returned = (pd.read_json(StringIO(requests.get(itemRequest).text)))
    resolvedName = returned.curie
    resolvedLabel = returned.label
    return resolvedName, resolvedLabel

def getCurie_Drug(name):
    itemRequest = 'https://name-resolution-sri.renci.org/lookup?string=' + name + '&autocomplete=false&offset=0&limit=10&biolink_type=ChemicalOrDrugOrTreatment'
    returned = (pd.read_json(StringIO(requests.get(itemRequest).text)))
    resolvedName = returned.curie
    resolvedLabel = returned.label
    return resolvedName[0], resolvedLabel[0]

def build_string_from_list(list):
    outString = "["
    for item in list:
        outString += item + ", "
    outString = outString[:-2] + "]"
    return outString

async def getAllDiseaseCuries(names: list[str]) -> list[str]:
    async with aiohttp.ClientSession() as session:
        tasks = [getCurie_Disease_async(session, name) for name in names]

        results = []
        for f in tqdm.asyncio.tqdm.as_completed(tasks):
            results.append(await f)
            results = await asyncio.gather(*tasks)
    return results
        
async def main():
    diseaseData = pd.read_excel('../active_ingredients_to_structured_lists_v2.xlsx')
    diseaseLabelList = []
    diseaseIDList = []
    diseaseList = []
    drugList = []
    drugIDList = []
    drugLabelList = []
    
    nRows = len(list(diseaseData['Structured Disease list']))
    
    labelDict = {}
    idDict = {}
        
    
    print("creating tasks")
    for index, row in tqdm.tqdm(diseaseData.iterrows(), total=len(diseaseData)):
        curr_row_diseasesTreated = row['Structured Disease list']
        
        if type(curr_row_diseasesTreated)!=float:
            curr_row_drugsInTherapy = row['Active Ingredients']
            curr_row_disease_ids = []
            curr_row_disease_id_labels = []
            curr_row_diseaseList = curr_row_diseasesTreated.replace("[","").replace("]","").replace('\'','').split(',')
            
            #try:
                #drugCurie,drugLabel = getCurie_Drug(curr_row_drugsInTherapy)
                #drugID = drugCurie[0]
                #drugIDLabel = drugLabel[0]
                    
            #except:
               # print("could not identify drug: ", curr_row_drugsInTherapy)
                #drugID = "NameRes Failed"
                #drugIDLabel = "NameRes Failed"
    
            for idx2,item in enumerate(curr_row_diseaseList):
                
                item = item.strip().upper().replace(" \n","").replace(" (PREVENTATIVE)","")
                #curr_row_diseaseList[idx2] = item
                #print(item)
                #try:
                    #print(item)
                    #diseaseCurie,diseaseLabel = getCurie_Disease(item)
                    #diseaseIDList.append(diseaseCurie[0])
                    #diseaseLabelList.append(diseaseLabel[0])
                diseaseList.append(item)
                    #drugList.append(curr_row_drugsInTherapy)
                    #drugIDList.append(drugID)
                    #drugLabelList.append(drugIDLabel)
                    
                #except:
                    #print("error during name resolving")


    curies, labels = getAllDiseaseCuries(diseaseList)
    print(curies)

    
    
    data = pd.DataFrame({
        "active ingredients": drugList,
        "drug ID": drugIDList,
        "drug label": drugLabelList,
        "disease list": diseaseList,
        "disease curie": diseaseIDList,
        "disease label": diseaseLabelList,
    })
    data.to_excel("contraindicationList.xlsx")


asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop